# Data cleaning/mining and feature engineering

In [1]:
%matplotlib inline
# from IPython.core.interactiveshell import InteractiveShell
# InteractiveShell.ast_node_interactivity = "all"
from IPython.display import display

# Std Lib
import re
# from datetime import date
from datetime import datetime

# 3rd Party
import pandas as pd
import pylab as plt
import seaborn as sns

In [2]:
!ls

Baltimore_City_Employee_Salaries_FY2012.csv
Baltimore_City_Employee_Salaries_FY2013.csv
Baltimore_City_Employee_Salaries_FY2014.csv
Baltimore_City_Employee_Salaries_FY2015.csv
Baltimore_City_Employee_Salaries_FY2016.csv
Data_mining.ipynb
README.md
all.csv


## Data Cleaning

In [3]:
# data = pd.read_csv('Baltimore_City_Employee_Salaries_FY2015.csv')

# file_names = ['Baltimore_City_Employee_Salaries_FY201%s.csv'%x for x in range(2, 7)]
# # display(data)

# data = pd.DataFrame()
# for furl in file_names:
#     data = data.append(pd.read_csv(furl), ignore_index=True)

data = pd.read_csv('all.csv')

In [4]:
# for now just drop the na values that are in the gross pay. we can find an appropriate value for them later
data = data.dropna()

In [5]:
data.head()

,﻿SurveyYear,Name,JobTitle,AgencyID,Agency,HireDate,AnnualSalaryMoney,GrossPayMoney,AnnualSalary,GrossPay
0,2012,"Aaron,Patricia G",Facilities/Office Services II,A03031,OED-Employment Dev,10/24/79,"$51,862.00","$52,247.39",51862.0,52247.39
1,2012,"Aaron,Petra L",ASSISTANT STATE'S ATTORNEY,A29005,States Attorneys Office,9/25/06,"$64,000.00","$59,026.81",64000.0,59026.81
2,2012,"Abaineh,Yohannes T",EPIDEMIOLOGIST,A65026,HLTH-Health Department,7/23/09,"$57,900.00","$57,129.79",57900.0,57129.79
3,2012,"Abdal-Rahim,Naim A",EMT FIREFIGHTER,A64215,Fire Department,3/30/11,"$34,146.00","$35,537.88",34146.0,35537.88
4,2012,"Abdi,Ezekiel W",POLICE OFFICER,A99398,Police Department,6/14/07,"$58,244.00","$62,669.25",58244.0,62669.25


In [6]:
data.dtypes

﻿SurveyYear            int64
Name                  object
JobTitle              object
AgencyID              object
Agency                object
HireDate              object
AnnualSalaryMoney     object
GrossPayMoney         object
AnnualSalary         float64
GrossPay             float64
dtype: object

In [7]:
# data.SurveyYear = pd.to_numeric(data.SurveyYear)
data.HireDate = pd.to_datetime(data.HireDate)

In [9]:
data.dtypes

﻿SurveyYear                   int64
Name                         object
JobTitle                     object
AgencyID                     object
Agency                       object
HireDate             datetime64[ns]
AnnualSalaryMoney            object
GrossPayMoney                object
AnnualSalary                float64
GrossPay                    float64
dtype: object

In [10]:
# convert all the financial data to floats
# x = lambda x: re.findall(r'[^$]*', x)[1]
# data['AnnualSalaryFloat'] = data.AnnualSalary.apply(x)
# data.AnnualSalaryFloat = pd.to_numeric(data.AnnualSalaryFloat)

# data['GrossPayFloat'] = data.GrossPay.apply(x)
# data.GrossPayFloat = pd.to_numeric(data.GrossPayFloat)

In [11]:
data.head()

,﻿SurveyYear,Name,JobTitle,AgencyID,Agency,HireDate,AnnualSalaryMoney,GrossPayMoney,AnnualSalary,GrossPay
0,2012,"Aaron,Patricia G",Facilities/Office Services II,A03031,OED-Employment Dev,1979-10-24,"$51,862.00","$52,247.39",51862.0,52247.39
1,2012,"Aaron,Petra L",ASSISTANT STATE'S ATTORNEY,A29005,States Attorneys Office,2006-09-25,"$64,000.00","$59,026.81",64000.0,59026.81
2,2012,"Abaineh,Yohannes T",EPIDEMIOLOGIST,A65026,HLTH-Health Department,2009-07-23,"$57,900.00","$57,129.79",57900.0,57129.79
3,2012,"Abdal-Rahim,Naim A",EMT FIREFIGHTER,A64215,Fire Department,2011-03-30,"$34,146.00","$35,537.88",34146.0,35537.88
4,2012,"Abdi,Ezekiel W",POLICE OFFICER,A99398,Police Department,2007-06-14,"$58,244.00","$62,669.25",58244.0,62669.25


In [12]:
data.dtypes

﻿SurveyYear                   int64
Name                         object
JobTitle                     object
AgencyID                     object
Agency                       object
HireDate             datetime64[ns]
AnnualSalaryMoney            object
GrossPayMoney                object
AnnualSalary                float64
GrossPay                    float64
dtype: object

In [13]:
# this one normalizes 'Agency' column when it = 'Mayors Office' to be 'Mayor's Office'
data.loc[data.loc[:,'Agency'].str.contains('Mayors Office') == True, 'Agency'] = data[data.Agency.str.contains('Mayors Office') == True].Agency.str.replace('Mayors Office', 'Mayor\'s Office')

In [14]:
data.Agency = data.Agency.str.strip()

## Feature Engineering

### Tenure

In [15]:
# calculates the tenure at the time the data was collected
today = pd.datetime(2015, 5, 30)
x = lambda x: today - x
data['Tenure'] = data.HireDate.apply(x)

# creates a feature of tenure in years stored as an int
x = lambda x: int(x / pd.Timedelta(1, 'Y'))
data['TenureYears'] = data.Tenure.apply(x)

In [16]:
# https://www.disabled-world.com/calculators-charts/wedding-Anniversaries.php
# Wooden (5th), Tin (10th), Crystal (15th), China (20th), Silver (25th), Pearl (30th), Ruby (40th), Golden (50th), and Diamond (60th)
def tenure_buckets(tenure_years):
    if tenure_years > 60:
        return 'Diamond(>60)'
    elif tenure_years > 50:
        return 'Gold(>50)'
    elif tenure_years > 40:
        return 'Ruby(>40)'
    elif tenure_years > 30:
        return 'Pearl(>30)'
    elif tenure_years > 25:
        return 'Silver(>25)'
    elif tenure_years > 20:
        return 'China(>20)'
    elif tenure_years > 15:
        return 'Crystal(>15)'
    elif tenure_years > 10:
        return 'Tin(>10)'
    elif tenure_years > 5:
        return 'Wood(>5)'
    else:
        return 'Paper(<5)'

tenure_bucket_order = ['Paper(<5)', 'Wood(>5)', 'Tin(>10)', 'Crystal(>15)', 'China(>20)',
                           'Silver(>25)', 'Pearl(>30)', 'Ruby(>40)', 'Gold(>50)', 'Diamond(>60)']

def tenure_buckets_rank(tenure_years):
    if tenure_years > 60:
        return 1
    elif tenure_years > 50:
        return 2
    elif tenure_years > 40:
        return 3
    elif tenure_years > 30:
        return 4
    elif tenure_years > 25:
        return 5
    elif tenure_years > 20:
        return 6
    elif tenure_years > 15:
        return 7
    elif tenure_years > 10:
        return 8
    elif tenure_years > 5:
        return 9
    else:
        return 10

In [17]:
data['TenureClass'] = data.TenureYears.apply(tenure_buckets)
data['TenureClassRank'] = data.TenureYears.apply(tenure_buckets_rank)

### Agency categories/subcategory

In [18]:
# first 3 of the AgencyID is the AgencyCode
x = lambda x: x[:-3]
data['AgencyCode'] = data.AgencyID.apply(x)

# first 3 of the AgencyID is the AgencySubCode
x = lambda x: x[-3:]
data['AgencySubCode'] = data.AgencyID.apply(x)

In [19]:
# this crazy regex is to strip the trailing code data at the end of Agency
# https://regex101.com/r/9F56Wh/1
x = lambda x: re.sub(r' \([\w-]+\) *\(*[\w]*\)*|-Hea[lt]{2}h.*| \(.*| \d+', '', x)
data['AgencyCat'] = data.Agency.apply(x)

In [20]:
data.head()

,﻿SurveyYear,Name,JobTitle,AgencyID,Agency,HireDate,AnnualSalaryMoney,GrossPayMoney,AnnualSalary,GrossPay,Tenure,TenureYears,TenureClass,TenureClassRank,AgencyCode,AgencySubCode,AgencyCat
0,2012,"Aaron,Patricia G",Facilities/Office Services II,A03031,OED-Employment Dev,1979-10-24,"$51,862.00","$52,247.39",51862.0,52247.39,13002 days,35,Pearl(>30),4,A03,031,OED-Employment Dev
1,2012,"Aaron,Petra L",ASSISTANT STATE'S ATTORNEY,A29005,States Attorneys Office,2006-09-25,"$64,000.00","$59,026.81",64000.0,59026.81,3169 days,8,Wood(>5),9,A29,005,States Attorneys Office
2,2012,"Abaineh,Yohannes T",EPIDEMIOLOGIST,A65026,HLTH-Health Department,2009-07-23,"$57,900.00","$57,129.79",57900.0,57129.79,2137 days,5,Paper(<5),10,A65,026,HLTH
3,2012,"Abdal-Rahim,Naim A",EMT FIREFIGHTER,A64215,Fire Department,2011-03-30,"$34,146.00","$35,537.88",34146.0,35537.88,1522 days,4,Paper(<5),10,A64,215,Fire Department
4,2012,"Abdi,Ezekiel W",POLICE OFFICER,A99398,Police Department,2007-06-14,"$58,244.00","$62,669.25",58244.0,62669.25,2907 days,7,Wood(>5),9,A99,398,Police Department


In [25]:
data.pivot_table(index=['AgencyCode', 'AgencyCat'], aggfunc=pd.Series.nunique, values=['Agency','Name'])

,,Agency,Name
AgencyCode,AgencyCat,,
,OED-Employment Dev,1,1
A01,Mayor's Office,4,200
A02,City Council,5,141
A03,OED-Employment Dev,19,268
A04,R&P-Recreation,11,214
A06,Housing & Community Dev,34,739
A08,M-R Human Services,13,206
A09,Liquor License Board,3,53
A12,FIN-Acct & Payroll,4,75


### First/Last Name, MI and Suffix

In [20]:
data.to_clipboard()

KeyboardInterrupt: 

In [21]:
x = lambda x: x.split(',')[1].split(' ')[0]
data['FirstName'] = data.Name.apply(x)

x = lambda x: x.split(',')[1].split(' ')[1] if len(x.split(',')[1].split(' ')) > 1 else ''
data['MiddleInitial'] = data.Name.apply(x)

x = lambda x: x.split(',')[0]
data['LastName'] = data.Name.apply(x)

IndexError: list index out of range

In [ ]:
# LastName Sr's & Jr's account for 700+ employees, lets clean that up
    #this is a naive selector.. (counter example Sriam)
    # data[data['Name'].str.contains('Jr') | data['LastName'].str.contains('Sr')]

# https://regex101.com/r/OIewK9/1
# first we are pulling out the suffix into a new field
x = lambda x: re.findall(r'[JS]r[\.]?$', x)[0] if len(re.findall(r'[JS]r[\.]?$', x)) > 0 else ''
data['SuffixName'] = data.LastName.apply(x)

# then we are replacing the instance inside of the Last name field
x = lambda x: re.sub(r'[JS]r[\.]?$', '', x)
data.LastName = data.LastName.apply(x)

### Wages and Approx. hours

In [ ]:
def salary_buckets(salary):
    if salary > 150 * 10**3:
        return '>150K'
    elif salary > 125 * 10**3:
        return '>125K'
    elif salary > 100 * 10**3:
        return '>100K'
    elif salary > 75 * 10**3:
        return '>75K'
    elif salary > 50 * 10**3:
        return '>50K'
    elif salary > 25 * 10**3:
        return '>25K'
    else:
        return '<25K'

salary_bucket_order = ['<25K', '>25K', '>50K', '>75K', '>100K', '>125K', '>150K']    

def salary_bucket_ranks(salary):
    if salary > 150 * 10**3:
        return 1
    elif salary > 125 * 10**3:
        return 2
    elif salary > 100 * 10**3:
        return 3
    elif salary > 75 * 10**3:
        return 4
    elif salary > 50 * 10**3:
        return 5
    elif salary > 25 * 10**3:
        return 7
    else:
        return 8

In [ ]:
data['AnnualSalaryBucket'] = data.AnnualSalaryFloat.apply(salary_buckets)
data['AnnualSalaryBucketRank'] = data.AnnualSalaryFloat.apply(salary_bucket_ranks)

data['GrossPayBucket'] = data.GrossPayFloat.apply(salary_buckets)
data['GrossPayBucketRank'] = data.GrossPayFloat.apply(salary_bucket_ranks)

In [ ]:
# approximate hourly rate
x = lambda x: x/2080
data['ApproxHourlyRate'] = data.AnnualSalaryFloat.apply(x)

In [ ]:
# try to approximate hours worked
# if GrossPay > AnnualSalary:
#     2080 + (GrossPay - AnnualSalary) / (ApproxHourlyRate * 1.5)
# else:
#     (GrossPay / ApproxHourlyRate)

def approx_hours_worked(row):
    if row['GrossPayFloat'] > row['AnnualSalaryFloat']:
        return 2080 + (row['GrossPayFloat'] - row['AnnualSalaryFloat']) / (row['ApproxHourlyRate'] * 1.5)
    else:
        return row['GrossPayFloat'] / row['ApproxHourlyRate']

In [ ]:
data['ApproxHoursWorked'] = data.apply(approx_hours_worked, axis=1)

In [ ]:
data['ApproxAvgWeeklyHours'] = data.ApproxHoursWorked / 52

In [ ]:
data.head()

### Employee Type (FTE, PTE, Hourly, seasonal)

In [ ]:
# gotta initialize the column so values dont start as NaN
data['EmployeeType'] = ''

In [ ]:
# sets employee type to 'Hourly' if JobTitle contains hourly
data.loc[data.loc[:,'JobTitle'].str.contains('Hourly') == True, 'EmployeeType'] = 'Hourly'

In [ ]:
# sets employee type to 'Hourly' if Agency contains hourly
data.loc[data.loc[:,'Agency'].str.contains('part-time') == True, 'EmployeeType'] = 'Part-Time'

In [ ]:
# sets employee type to 'Seasonal' if JobTitle contains seasonal
data.loc[data.loc[:,'JobTitle'].str.contains('SEASONAL') == True, 'EmployeeType'] = 'Seasonal'

In [ ]:
# if employee type is blank assign 'Full-Time'
x = lambda x: 'Full-Time' if x == '' else x
data.EmployeeType = data.EmployeeType.apply(x)

In [ ]:
data.pivot_table(index='EmployeeType', aggfunc='mean')

### Job Title level

In [ ]:
# creating a feature for the Level of the Job Title
# r'I+$' looks for I one or more times at the end of the string
x = lambda x: len(re.findall(r'I+$', x)[0]) if len(re.findall(r'I+$', x)) > 0 else 'NaN'
data['JobTitleLevel'] = data.JobTitle.apply(x)

In [ ]:
data.pivot_table(index='JobTitleLevel', aggfunc='count')

## Interesting data points

In [ ]:
data[data.AnnualSalaryBucketRank - data.GrossPayBucketRank > 2].head()

In [ ]:
data[data.TenureYears > 50]

In [ ]:
sns.stripplot(y='GrossPayBucket', x='ApproxAvgWeeklyHours', data= data, jitter=True,
              order=salary_bucket_order, hue='AnnualSalaryBucket', 
              hue_order= salary_bucket_order, orient='h')

In [ ]:
data[data.ApproxAvgWeeklyHours < 10]

In [ ]:
sns.swarmplot(y='GrossPayBucket', x='ApproxAvgWeeklyHours', data= data, 
              order=salary_bucket_order, hue='AnnualSalaryBucket', 
              hue_order= salary_bucket_order, orient='h')